# Jupyter Notebook to train MinimalSam for MAX78000 on COLAB

### Change Python Version

In [ ]:
!sudo apt-get install python3.11
!sudo apt-get install python3.11-venv
!sudo apt install python3.11-dev
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
!sudo update-alternatives --set python3 /usr/bin/python3.11
!python --version
!python3 -m ensurepip

### Install ai8x-training

In [ ]:
!sudo apt-get install -y make build-essential libssl-dev zlib1g-dev \
  libbz2-dev libreadline-dev libsqlite3-dev wget curl llvm \
  libncurses5-dev libncursesw5-dev xz-utils tk-dev libffi-dev liblzma-dev \
  libsndfile-dev portaudio19-dev libsox-dev

!git clone --recursive https://github.com/analogdevicesinc/ai8x-training.git

In [ ]:
!pip3 install -U wheel
!cd ai8x-training && pip3 install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 #Note: change pyffmpeg==2.5.1 in requirements.txt

### Download Dataset

In [ ]:
%%bash

cd ai8x-training
mkdir data
cd data

echo "Downloading COCO 2017 dataset..."
curl -s -L -O http://images.cocodataset.org/zips/val2017.zip &
curl -s -L -O http://images.cocodataset.org/zips/train2017.zip &
curl -s -L -O http://images.cocodataset.org/annotations/annotations_trainval2017.zip &
wait
unzip -q val2017.zip && rm val2017.zip &
unzip -q train2017.zip && rm train2017.zip &
unzip -q annotations_trainval2017.zip && rm annotations_trainval2017.zip &
wait
echo "✅ Dataset successfully downloaded"

### Deploy configuration files (manually)

### Train

In [ ]:
!cd ai8x-training && python train.py --deterministic --pr-curves --epochs 20 --optimizer Adam --lr 0.001 --wd 0.0001--model minimalsam --use-bias --dataset minimalsam_dataset --device MAX78000 --batch-size 32 --qat-policy policies/qat_policy_minimalsam.yaml --compress policies/schedule_minimalsam.yaml --validation-split 0 --print-freq 250 "$@"
#python train.py --deterministic --pr-curves --epochs 200 --optimizer Adam --lr 0.001 --wd 0 --model ai85unetlarge --out-fold-ratio 4 --use-bias --dataset AISegment_352 --device MAX78000 --batch-size 32 --qat-policy policies/qat_policy_aisegment.yaml --compress policies/schedule-aisegment.yaml --validation-split 0 --print-freq 250 "-


# Download logs

In [ ]:
!zip -r logs.zip ai8x-training/logs

from google.colab import files
files.download("logs.zip")